In [1]:
import pandas as pd
import sqlite3
from sqlite3 import Error

In [2]:
def create_connection():
    conn = None;
    try:
        conn = sqlite3.connect('ventas.db')
        print(f'Connection succeeded {sqlite3.version}')
    except Error as e:
            print(f'Error {e} in the connection')
    return conn

In [3]:
def create_table(conn):
    try:
        query = '''
            CREATE TABLE ventas (
            ID INTEGER PRIMARY KEY,
            Vendedor TEXT NOT NULL,
            Producto TEXT NOT NULL,
            Cantidad INTEGER,
            Fecha_venta DATE NOT NULL
            );
        '''
        conn.execute(query)
        print('Table created')
    except Error as e:
        print(f'Error {e} in table creation')

In [4]:
def insert_data(conn):
    try:
        query = '''
            INSERT INTO ventas(ID, Vendedor, Producto, Cantidad, Fecha_venta)
            VALUES
            (1, 'Ana', 'Libro', 10, '2023-01-01'),
            (2, 'Beto', 'Lápiz', 30, '2023-01-05'),
            (3, 'Carlos', 'Cuaderno', 15, '2023-01-08'),
            (4, 'Ana', 'Cuaderno', 20, '2023-01-09'),
            (5, 'Beto', 'Esfero', 50, '2023-01-12'),
            (6, 'Carlos', 'Libro', 25, '2023-01-15'),
            (7, 'Ana', 'Esfero', 30, '2023-01-17'),
            (8, 'Beto', 'Cuaderno', 40, '2023-01-19'),
            (9, 'Carlos', 'Lápiz', 35, '2023-01-22'),
            (10, 'Ana', 'Libro', 45, '2023-01-25');
        '''
        conn.execute(query)
        conn.commit()
        print('Data inserted successfully')
    except Error as e:
        print(f'Error {e} in data insertion')

In [5]:
conn = create_connection()

with conn: 
    create_table(conn)
    insert_data(conn)

Connection succeeded 2.6.0
Table created
Data inserted successfully


In [6]:
def execute_query(conn, query):
    try:
        df = pd.read_sql(query, conn)
        display(df)
    except Error as e:
        print(f'Error {e} in the query execution')

In [10]:
conn = create_connection()

with conn:
    print ('Query 1: All lines')
    query1 = '''
    SELECT *
    FROM ventas;
    '''
    execute_query(conn, query1)

Connection succeeded 2.6.0
Query 1: All lines


,ID,Vendedor,Producto,Cantidad,Fecha_venta
0,1,Ana,Libro,10,2023-01-01
1,2,Beto,Lápiz,30,2023-01-05
2,3,Carlos,Cuaderno,15,2023-01-08
3,4,Ana,Cuaderno,20,2023-01-09
4,5,Beto,Esfero,50,2023-01-12
5,6,Carlos,Libro,25,2023-01-15
6,7,Ana,Esfero,30,2023-01-17
7,8,Beto,Cuaderno,40,2023-01-19
8,9,Carlos,Lápiz,35,2023-01-22
9,10,Ana,Libro,45,2023-01-25


In [11]:
with conn:
    print ('Query 2: OVER Function')
    query2 = '''
    SELECT Vendedor, Producto, Cantidad,
    SUM(Cantidad) OVER() AS Cantidad_total
    FROM ventas;
    '''
    execute_query(conn, query2)

Query 2: OVER Function


,Vendedor,Producto,Cantidad,Cantidad_total
0,Ana,Libro,10,300
1,Beto,Lápiz,30,300
2,Carlos,Cuaderno,15,300
3,Ana,Cuaderno,20,300
4,Beto,Esfero,50,300
5,Carlos,Libro,25,300
6,Ana,Esfero,30,300
7,Beto,Cuaderno,40,300
8,Carlos,Lápiz,35,300
9,Ana,Libro,45,300


In [12]:
with conn:
    print ('Query 3: OVER Function')
    query3 = '''
    SELECT Vendedor, Producto, Cantidad,
    MAX(Cantidad) OVER() AS Maximo_total
    FROM ventas;
    '''
    execute_query(conn, query3)

Query 3: OVER Function


,Vendedor,Producto,Cantidad,Maximo_total
0,Ana,Libro,10,50
1,Beto,Lápiz,30,50
2,Carlos,Cuaderno,15,50
3,Ana,Cuaderno,20,50
4,Beto,Esfero,50,50
5,Carlos,Libro,25,50
6,Ana,Esfero,30,50
7,Beto,Cuaderno,40,50
8,Carlos,Lápiz,35,50
9,Ana,Libro,45,50


In [14]:
with conn:
    print ('Query 4: PARTITION BY Function')
    query4 = '''
    SELECT Vendedor, Producto, Cantidad,
    SUM(Cantidad) OVER(PARTITION BY Vendedor) AS Cantidad_total_vendedor
    FROM ventas;
    '''
    execute_query(conn, query4)

Query 4: PARTITION BY Function


,Vendedor,Producto,Cantidad,Cantidad_total_vendedor
0,Ana,Libro,10,105
1,Ana,Cuaderno,20,105
2,Ana,Esfero,30,105
3,Ana,Libro,45,105
4,Beto,Lápiz,30,120
5,Beto,Esfero,50,120
6,Beto,Cuaderno,40,120
7,Carlos,Cuaderno,15,75
8,Carlos,Libro,25,75
9,Carlos,Lápiz,35,75


In [16]:
with conn:
    print ('Query 5: PARTITION BY Function')
    query4 = '''
    SELECT Vendedor, Producto, Cantidad,
    AVG(Cantidad) OVER(PARTITION BY Vendedor) AS Media_vendedor
    FROM ventas
    ORDER BY Media_vendedor DESC;
    '''
    execute_query(conn, query4)

Query 5: PARTITION BY Function


,Vendedor,Producto,Cantidad,Media_vendedor
0,Beto,Lápiz,30,40.00
1,Beto,Esfero,50,40.00
2,Beto,Cuaderno,40,40.00
3,Ana,Libro,10,26.25
4,Ana,Cuaderno,20,26.25
5,Ana,Esfero,30,26.25
6,Ana,Libro,45,26.25
7,Carlos,Cuaderno,15,25.00
8,Carlos,Libro,25,25.00
9,Carlos,Lápiz,35,25.00


In [17]:
with conn:
    print ('Query 6: RANK Function (accepts ties)')
    query6 = '''
    SELECT Vendedor, Producto, Cantidad,
    RANK() OVER(ORDER BY Cantidad DESC) AS Ranking
    FROM ventas;
    '''
    execute_query(conn, query6)

Query 6: RANK Function (accepts ties)


,Vendedor,Producto,Cantidad,Ranking
0,Beto,Esfero,50,1
1,Ana,Libro,45,2
2,Beto,Cuaderno,40,3
3,Carlos,Lápiz,35,4
4,Beto,Lápiz,30,5
5,Ana,Esfero,30,5
6,Carlos,Libro,25,7
7,Ana,Cuaderno,20,8
8,Carlos,Cuaderno,15,9
9,Ana,Libro,10,10


In [19]:
with conn:
    print ('Query 7: RANK Function (accepts ties)')
    query7 = '''
    SELECT Vendedor, Producto, Cantidad,
    RANK() OVER(PARTITION BY Vendedor ORDER BY Cantidad DESC) AS Ranking
    FROM ventas;
    '''
    execute_query(conn, query7)

Query 7: RANK Function (accepts ties)


,Vendedor,Producto,Cantidad,Ranking
0,Ana,Libro,45,1
1,Ana,Esfero,30,2
2,Ana,Cuaderno,20,3
3,Ana,Libro,10,4
4,Beto,Esfero,50,1
5,Beto,Cuaderno,40,2
6,Beto,Lápiz,30,3
7,Carlos,Lápiz,35,1
8,Carlos,Libro,25,2
9,Carlos,Cuaderno,15,3


In [21]:
with conn:
    print ('Query 8: GROUP BY')
    query8 = '''
    SELECT Vendedor, SUM(Cantidad)
    FROM ventas
    GROUP BY Vendedor;
    '''
    execute_query(conn, query8)

Query 8: GROUP BY


,Vendedor,SUM(Cantidad)
0,Ana,105
1,Beto,120
2,Carlos,75


In [23]:
with conn:
    print ('Query 8.1: RANK Function')
    query8 = '''
    SELECT Vendedor, SUM(Cantidad),
    RANK() OVER (ORDER BY SUM(Cantidad) DESC) AS Rank
    FROM ventas
    GROUP BY Vendedor;
    '''
    execute_query(conn, query8)

Query 8.1: RANK Function


,Vendedor,SUM(Cantidad),Rank
0,Beto,120,1
1,Ana,105,2
2,Carlos,75,3


In [28]:
with conn:
    print ('Query 9: LAG Function')
    query9 = '''
    SELECT Vendedor, Producto, Fecha_venta, Cantidad,
    LAG(Cantidad) OVER(PARTITION BY Vendedor ORDER BY Fecha_venta) AS Cantidad_anterior
    FROM ventas;
    '''
    execute_query(conn, query9)

Query 9: LAG Function


,Vendedor,Producto,Fecha_venta,Cantidad,Cantidad_anterior
0,Ana,Libro,2023-01-01,10,NaN
1,Ana,Cuaderno,2023-01-09,20,10.0
2,Ana,Esfero,2023-01-17,30,20.0
3,Ana,Libro,2023-01-25,45,30.0
4,Beto,Lápiz,2023-01-05,30,NaN
5,Beto,Esfero,2023-01-12,50,30.0
6,Beto,Cuaderno,2023-01-19,40,50.0
7,Carlos,Cuaderno,2023-01-08,15,NaN
8,Carlos,Libro,2023-01-15,25,15.0
9,Carlos,Lápiz,2023-01-22,35,25.0


In [29]:
with conn:
    print ('Query 10: LAG Function')
    query10 = '''
    SELECT Vendedor, Producto, Fecha_venta, Cantidad,
    LEAD(Cantidad) OVER(PARTITION BY Vendedor ORDER BY Fecha_venta) AS Cantidad_anterior
    FROM ventas;
    '''
    execute_query(conn, query10)

Query 10: LAG Function


,Vendedor,Producto,Fecha_venta,Cantidad,Cantidad_anterior
0,Ana,Libro,2023-01-01,10,20.0
1,Ana,Cuaderno,2023-01-09,20,30.0
2,Ana,Esfero,2023-01-17,30,45.0
3,Ana,Libro,2023-01-25,45,NaN
4,Beto,Lápiz,2023-01-05,30,50.0
5,Beto,Esfero,2023-01-12,50,40.0
6,Beto,Cuaderno,2023-01-19,40,NaN
7,Carlos,Cuaderno,2023-01-08,15,25.0
8,Carlos,Libro,2023-01-15,25,35.0
9,Carlos,Lápiz,2023-01-22,35,NaN


In [30]:
conn.close()